In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from src.metrics import *
from settings import *
from src.data import generate

instruments = ['Guitar', 'Accordion', 'Violin']

# from setup_logging import setup_logging
# setup_logging()

#generate.my_run(instruments)
datasets = [pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED_NORMALIZED, f'data_{instrument.lower()}.pkl')) for instrument in instruments]
datasets_augmented = [pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED_NORMALIZED, f'data_{instrument.lower()}.pkl')) for instrument in instruments]

2023-05-19 13:43:20.537442: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 13:43:20.673879: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 13:43:21.280696: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-05-19 13:43:21.280795: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64]

In [2]:
print([dataset['spectrogram'][1].shape for dataset in datasets])

[(128, 87), (128, 87), (128, 87)]


In [3]:
import random
import keras
import os, glob
import logging
import librosa, librosa.display

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential, model_from_json
from keras import backend as K

from src.metrics import *
from settings import *
from src.data import generate
from src.processing import *
from src.model import CNN
from src.data.preprocessing import get_most_shape
from setup_logging import setup_logging

setup_logging()
logger = logging.getLogger('src.train')

augmented = False

train_datas = []
test_datas = []

for dataset in datasets:
    train_data, test_data = train_test_split(dataset, augmented=augmented, split_ratio=0.8)
    train_datas.append(train_data)
    test_datas.append(test_data)

train_data = pd.concat(train_datas)
test_data = pd.concat(test_datas)

most_shape = get_most_shape(train_data)

logger.info(f"Number of train samples: {len(train_data)}")
logger.info(f"Number of test samples: {len(test_data)}")
# most_shape = get_most_shape(dataset)
#train_data, test_data = train_test_split(dataset, augmented=augmented, split_ratio=0.65)

X_train, y_train = features_target_split(train_data)
X_test, y_test = features_target_split(test_data)

# Reshape for CNN input
X_train, X_test = reshape_feature_CNN(X_train), reshape_feature_CNN(X_test)

# Preserve y_test values
y_test_values = y_test.copy()

# One-Hot encoding for classes
y_train, y_test = one_hot_encode(y_train), one_hot_encode(y_test)

# Instance of CNN model
cnn = CNN(most_shape)
logger.info(str(cnn))

cnn.train(X_train, y_train, X_test, y_test)
cnn.evaluate(X_train, y_train, X_test, y_test)

if tf.__version__ != '1.8.0':
    predict_x=cnn.model.predict(X_test)
    predictions = np.argmax(predict_x,axis=1)
else:    
    predictions = cnn.model.predict_classes(X_test)
conf_matrix=confusion_matrix(y_test_values, predictions, labels=range(10))
logger.info('Confusion Matrix for classes {}:\n{}'.format(CLASSES, conf_matrix))
cnn.save_model(name="model_nopiano_noaugment_2")

src.processing - INFO - Start train test split with split ratio: 0.8
src.processing - INFO - Number of training samples is 1600
src.processing - INFO - Number of testing samples is 400
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.8
src.processing - INFO - Number of training samples is 80
src.processing - INFO - Number of testing samples is 20
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.8
src.processing - INFO - Number of training samples is 80
src.processing - INFO - Number of testing samples is 20
src.processing - INFO - Train test split completed
The most frequent shape is (128, 87)
src.train - INFO - Number of train samples: 1760
src.train - INFO - Number of test samples: 440
src.processing - INFO - Start feature target split
src.processing - INFO - Feature target split completed
src.processing - INFO - Start feature target split
src.proc

2023-05-19 13:43:39.165944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6


Epoch 1/67


2023-05-19 13:43:42.239176: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600
2023-05-19 13:43:43.012887: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


88/88 [==============================] - 6s 15ms/step - loss: 2.8050 - accuracy: 0.1068 - precision: 0.0667 - recall: 0.0063 - fmeasure: 0.0110 - val_loss: 2.2901 - val_accuracy: 0.1091 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_fmeasure: 0.0000e+00
Epoch 2/67
88/88 [==============================] - 1s 12ms/step - loss: 2.2753 - accuracy: 0.1500 - precision: 0.1080 - recall: 0.0063 - fmeasure: 0.0118 - val_loss: 2.2327 - val_accuracy: 0.1568 - val_precision: 0.0455 - val_recall: 0.0045 - val_fmeasure: 0.0083
Epoch 3/67
88/88 [==============================] - 1s 12ms/step - loss: 2.2557 - accuracy: 0.1676 - precision: 0.2254 - recall: 0.0176 - fmeasure: 0.0322 - val_loss: 2.2241 - val_accuracy: 0.1841 - val_precision: 0.0455 - val_recall: 0.0023 - val_fmeasure: 0.0043
Epoch 4/67
88/88 [==============================] - 1s 12ms/step - loss: 2.2103 - accuracy: 0.1937 - precision: 0.1875 - recall: 0.0131 - fmeasure: 0.0242 - val_loss: 2.1073 - val_accuracy: 0.2273 - val_p

In [4]:
dataset_piano = pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED_NORMALIZED, 'data_piano.pkl'))

test_data = dataset_piano

X_test = test_data['spectrogram']
X_test = np.array([np.pad(x, ((0, 0), (0, 87-x.shape[1])), 'constant') for x in dataset['spectrogram']])
y_test = test_data['class_ID']

X_test = np.array([x.reshape( (128, 87, 1) ) for x in X_test])

y_test_values=y_test
y_test = np.array(keras.utils.to_categorical(y_test, 10))

score = cnn.model.evaluate(X_test,y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', score[2])
print('Test recall:', score[3])
print('Test f1-score:', score[4])

4/4 [==============================] - 0s 15ms/step - loss: 0.3280 - accuracy: 0.9400 - precision: 0.9531 - recall: 0.9531 - fmeasure: 0.9531
Test loss: 0.32797059416770935
Test accuracy: 0.9399999976158142
Test precision: 0.953125
Test recall: 0.953125
Test f1-score: 0.9531249403953552


: 

# Baseline performance on piano

In [7]:
with open(MODEL_1_JSON, "r") as json_file:
	loaded_model_json = json_file.read()

model = model_from_json(loaded_model_json)
model.load_weights(MODEL_1_H5)

model.compile(
            optimizer="Adam",
            loss="categorical_crossentropy",
            metrics=['accuracy', precision, recall, fmeasure])

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', score[2])
print('Test recall:', score[3])
print('Test f1-score:', score[4])

4/4 [==============================] - 0s 6ms/step - loss: 255.6006 - accuracy: 0.2900 - precision: 0.4453 - recall: 0.4453 - fmeasure: 0.4453
Test loss: 255.60064697265625
Test accuracy: 0.28999999165534973
Test precision: 0.4453125
Test recall: 0.4453125
Test f1-score: 0.4453124403953552
